In [2]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import datetime as dt

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, desc

engine = create_engine("sqlite:///Resources/hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [3]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

# Create our session (link) from Python to the DB
session = Session(engine)

In [4]:
query_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print("Query date:", query_date)

precip = session.query(measurement.date, measurement.prcp).\
    filter(measurement.date > query_date).\
    order_by(measurement.date).all()
    
prcp_rows = [{"Date": result[0], "Precip": result[1]} for result in precip]
prcp_rows

Query date: 2016-08-23


[{'Date': '2016-08-24', 'Precip': 0.08},
 {'Date': '2016-08-24', 'Precip': 2.15},
 {'Date': '2016-08-24', 'Precip': 2.28},
 {'Date': '2016-08-24', 'Precip': None},
 {'Date': '2016-08-24', 'Precip': 1.22},
 {'Date': '2016-08-24', 'Precip': 2.15},
 {'Date': '2016-08-24', 'Precip': 1.45},
 {'Date': '2016-08-25', 'Precip': 0.08},
 {'Date': '2016-08-25', 'Precip': 0.08},
 {'Date': '2016-08-25', 'Precip': 0.0},
 {'Date': '2016-08-25', 'Precip': 0.0},
 {'Date': '2016-08-25', 'Precip': 0.21},
 {'Date': '2016-08-25', 'Precip': 0.06},
 {'Date': '2016-08-25', 'Precip': 0.11},
 {'Date': '2016-08-26', 'Precip': 0.0},
 {'Date': '2016-08-26', 'Precip': 0.03},
 {'Date': '2016-08-26', 'Precip': 0.02},
 {'Date': '2016-08-26', 'Precip': 0.04},
 {'Date': '2016-08-26', 'Precip': 0.0},
 {'Date': '2016-08-26', 'Precip': 0.01},
 {'Date': '2016-08-27', 'Precip': 0.0},
 {'Date': '2016-08-27', 'Precip': 0.18},
 {'Date': '2016-08-27', 'Precip': 0.02},
 {'Date': '2016-08-27', 'Precip': 0.0},
 {'Date': '2016-08-27'

In [11]:
all_stations = engine.execute('SELECT * FROM station').fetchall()
all_stations

station_dict = [{"ID": result[0],
                 "Station": result [1],
                 "Name": result[2],
                 "Latitude": result[3],
                 "Longitude": result[4],
                 "Elevation": result[5]} for result in all_stations]

station_dict

[{'ID': 1,
  'Station': 'USC00519397',
  'Name': 'WAIKIKI 717.2, HI US',
  'Latitude': 21.2716,
  'Longitude': -157.8168,
  'Elevation': 3.0},
 {'ID': 2,
  'Station': 'USC00513117',
  'Name': 'KANEOHE 838.1, HI US',
  'Latitude': 21.4234,
  'Longitude': -157.8015,
  'Elevation': 14.6},
 {'ID': 3,
  'Station': 'USC00514830',
  'Name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US',
  'Latitude': 21.5213,
  'Longitude': -157.8374,
  'Elevation': 7.0},
 {'ID': 4,
  'Station': 'USC00517948',
  'Name': 'PEARL CITY, HI US',
  'Latitude': 21.3934,
  'Longitude': -157.9751,
  'Elevation': 11.9},
 {'ID': 5,
  'Station': 'USC00518838',
  'Name': 'UPPER WAHIAWA 874.3, HI US',
  'Latitude': 21.4992,
  'Longitude': -158.0111,
  'Elevation': 306.6},
 {'ID': 6,
  'Station': 'USC00519523',
  'Name': 'WAIMANALO EXPERIMENTAL FARM, HI US',
  'Latitude': 21.33556,
  'Longitude': -157.71139,
  'Elevation': 19.5},
 {'ID': 7,
  'Station': 'USC00519281',
  'Name': 'WAIHEE 837.5, HI US',
  'Latitude': 21.45167,
  'L

In [13]:
tobs_activity = session.query(measurement.station,
    func.count(measurement.tobs)
    ).group_by(measurement.station
    ).order_by(func.count(measurement.station).desc())
    
most_active_tobs = tobs_activity[0]
tobs_station_name = most_active_tobs[0]

tobs2 = session.query(measurement.date, measurement.tobs).\
    filter(measurement.date > query_date).\
    filter(measurement.station == tobs_station_name).\
    order_by(measurement.date).all()
    
tobs_station_dict = [{"Station": most_active_tobs[0],
                      "Date": result[0], "TOBS": result[1]} for result in tobs2]


In [22]:
tobs_station_dict = [{"Station": most_active_tobs[0],
                      "Date": result[0], "TOBS": result[1]} for result in tobs2]
tobs_station_dict

[{'Station': 'USC00519281', 'Date': '2016-08-24', 'TOBS': 77.0},
 {'Station': 'USC00519281', 'Date': '2016-08-25', 'TOBS': 80.0},
 {'Station': 'USC00519281', 'Date': '2016-08-26', 'TOBS': 80.0},
 {'Station': 'USC00519281', 'Date': '2016-08-27', 'TOBS': 75.0},
 {'Station': 'USC00519281', 'Date': '2016-08-28', 'TOBS': 73.0},
 {'Station': 'USC00519281', 'Date': '2016-08-29', 'TOBS': 78.0},
 {'Station': 'USC00519281', 'Date': '2016-08-30', 'TOBS': 77.0},
 {'Station': 'USC00519281', 'Date': '2016-08-31', 'TOBS': 78.0},
 {'Station': 'USC00519281', 'Date': '2016-09-01', 'TOBS': 80.0},
 {'Station': 'USC00519281', 'Date': '2016-09-02', 'TOBS': 80.0},
 {'Station': 'USC00519281', 'Date': '2016-09-03', 'TOBS': 78.0},
 {'Station': 'USC00519281', 'Date': '2016-09-04', 'TOBS': 78.0},
 {'Station': 'USC00519281', 'Date': '2016-09-05', 'TOBS': 78.0},
 {'Station': 'USC00519281', 'Date': '2016-09-06', 'TOBS': 73.0},
 {'Station': 'USC00519281', 'Date': '2016-09-07', 'TOBS': 74.0},
 {'Station': 'USC00519281

In [17]:
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
tobs2 = session.query(measurement.date, measurement.tobs).\
    filter(measurement.date > query_date).\
    filter(measurement.station == tobs_station_name).\
    order_by(measurement.date).all()
    
tobs2

[('2016-08-24', 77.0),
 ('2016-08-25', 80.0),
 ('2016-08-26', 80.0),
 ('2016-08-27', 75.0),
 ('2016-08-28', 73.0),
 ('2016-08-29', 78.0),
 ('2016-08-30', 77.0),
 ('2016-08-31', 78.0),
 ('2016-09-01', 80.0),
 ('2016-09-02', 80.0),
 ('2016-09-03', 78.0),
 ('2016-09-04', 78.0),
 ('2016-09-05', 78.0),
 ('2016-09-06', 73.0),
 ('2016-09-07', 74.0),
 ('2016-09-08', 80.0),
 ('2016-09-09', 79.0),
 ('2016-09-10', 77.0),
 ('2016-09-11', 80.0),
 ('2016-09-12', 76.0),
 ('2016-09-13', 79.0),
 ('2016-09-14', 75.0),
 ('2016-09-15', 79.0),
 ('2016-09-16', 78.0),
 ('2016-09-17', 79.0),
 ('2016-09-18', 78.0),
 ('2016-09-19', 78.0),
 ('2016-09-20', 76.0),
 ('2016-09-21', 74.0),
 ('2016-09-22', 77.0),
 ('2016-09-23', 78.0),
 ('2016-09-24', 79.0),
 ('2016-09-25', 79.0),
 ('2016-09-26', 77.0),
 ('2016-09-27', 80.0),
 ('2016-09-28', 78.0),
 ('2016-09-29', 78.0),
 ('2016-09-30', 78.0),
 ('2016-10-01', 77.0),
 ('2016-10-02', 79.0),
 ('2016-10-03', 79.0),
 ('2016-10-04', 79.0),
 ('2016-10-05', 79.0),
 ('2016-10-

In [ ]:
def calc_temps(start_date, end_date):
    
    return session.query(func.min(measurement.tobs), func.avg(measurement.tobs), func.max(measurement.tobs)).\
        filter(measurement.date >= start_date).filter(measurement.date <= end_date).all()
        


In [ ]:
@app.route("/api/v1.0/justice-league/superhero/<superhero>")
def justice_league_by_superhero__name(superhero):
    """Fetch the Justice League character whose superhero matches
       the path variable supplied by the user, or a 404 if not."""

    canonicalized = superhero.replace(" ", "").lower()
    for character in justice_league_members:
        search_term = character["superhero"].replace(" ", "").lower()

        if search_term == canonicalized:
            return jsonify(character)

    return jsonify({"error": "Character not found."}), 404